In [23]:

from cards import CardDeck
from hand_rankings import getHandRanking, getPlayerHand, getPlayerHandRanking, hand_ranking_classes
import hand_rankings
import players
from players import Player
import prob_best_hand
from random import shuffle

In [24]:
class PokerGame():
    
        def __init__(self,num_players=5,rounds=10):
            self.rounds = rounds
            self.players = self.createPlayers(num_players)

            self.num_wins = [0]*len(self.players)

            self.playGame(rounds)

            self.resetGame()

            pass

        def resetGame(self):
            #Bit hacky - fix
            self.players[0].resetPlayers()
            pass

        def playGame(self,rounds):
            rounds = self.rounds
            for round in range(rounds):
                current_players = []
                for player in self.players:
                    if not player.isBankrupt():
                        current_players.append(player)

                new_round = PokerRound(current_players)
                winner = new_round.getWinner()
                self.changePlayerOrder()
                #self.updateNumWins(winner)
                #new_round = PokerRound()

        def changePlayerOrder(self):
            first_player = self.players[:1]
            other_players = self.players[1:]
            self.players = other_players + first_player
            return self.players

        def listWins(self):
            for player in self.players:
                print(player.getNumWins(), player.getCash())
                #self.num_wins[player.name-1] = player.getNumWins()
                
        def createPlayers(self, n=5):
            player_list = []
            for i in range(n):
                new_player = Player()
                player_list.append(new_player)
            return player_list
        
        def updateNumWins(self, player_index):
            self.num_wins[player_index] += 1
            pass 
        
        def getPlayers(self):
            return self.players
        
        def getNumWins(self):
            #Also reset the game
            self.resetGame()
            return self.num_wins

class PokerRound(PokerGame):

    def __init__(self, players):

        self.players = players

        if len(self.players) >= 2:

            #TODO: Check if player is bankrupt
            #If so, then remove from future processing

            self.cash_pool = 0

            deck = self.createDeck()

            self.dealToPlayers(self.players, deck)

            #Small and big blind
            self.blind(players[0], 1)
            self.blind(players[1], 2)

            #print("Round 1")
            self.dealToCommunity(deck)

            #self.getPlayerRankings(players, deck)

            #print("Round 2")
            self.updateCommunity(deck)

            #self.getPlayerRankings(players, deck)
            
            #print("Round 3")
            self.updateCommunity(deck)
            self.makeBets(self.players)

            self.getPlayerRankings(self.players, deck)

            self.winner = self.determineWinner(self.players, deck)

        else:
            if len(self.players) == 1:
                self.winner = self.players[0]
            else:
                self.winner = "Undetermined"

        pass

    def blind(self, player, n=0):
        player.betBlind(n)
        self.addToCashPool(n)
        pass

    def makeBets(self, players):
        for player in players:
            player_bet = player.getBet()
            self.addToCashPool(player_bet)
        pass
    
    def createDeck(self):
        return CardDeck()
    
    def removeBankrupt(self, players):
        for player in players:
            if player.isBankrupt():
                print(player)
                players.pop(player)
        pass
    
    def dealToPlayers(self, players, deck):
        for player in players:
            deck.dealToPlayer(player)
        pass

    def dealToCommunity(self, deck):
        deck.dealToCommunity(3)
        self.playerAccessToCommunityCards(self.players, deck)
        pass

    def updateCommunity(self, deck):
        deck.dealToCommunity()
        self.playerAccessToCommunityCards(self.players, deck)
        pass

    def getCommunityCards(self, deck):
        return deck.getCommunityCards()
    
    def playerAccessToCommunityCards(self, players, deck):
        for player in players:
            community_cards = self.getCommunityCards(deck)
            player.getCommunityCards(community_cards)
        pass

    def checkCardUniqueness(self, players, deck):
        for player in players:
            unique_cards = []
            for card in getPlayerHand(player, deck):
                if card not in unique_cards:
                    unique_cards.append(card)
                else:
                    print("Error: ", getPlayerHand(player, deck))

    def getPlayerRankings(self, players, deck):
        #For testing purposes only!
        player_rankings = []
        for player in players:

            #print("Player hands: ", [i.name for i in getPlayerHand(player, deck)])
            player_rankings.append(getPlayerHandRanking(player, deck))
            #print("Player ranking: ", getPlayerHandRanking(player, deck).__name__)

        #print("Rankings: ", [i.__name__ for i in player_rankings])
        return player_rankings
    
    def addToCashPool(self, num):
        self.cash_pool += num
        pass
    
    def determineWinner(self, players, deck):
        current_highest = hand_rankings.HighCard
        current_index = hand_ranking_classes.index(current_highest)
        winner = players[0]

        for player in players:
            player_rank = getPlayerHandRanking(player, deck)
            player_index = hand_ranking_classes.index(player_rank)
            if player_index < current_index:
                current_index = player_index
                winner = player
        
        winner.addWin()

        winner.addCash(self.cash_pool)

        return winner
    
    def getWinner(self):
        return self.winner
    


In [25]:
new_game = PokerGame(rounds=100)
new_game.listWins()

#Noticed a duplicate card in hand - hard to replicate but worth investigating
#Likely cause - player/community cards are not being removed from cards still in deck
#Need to investigate further 

#TODO: Investigate random walk/gambler's ruin

#TODO: Sometimes players go below 0 - investigate

35 212
3 0
18 0
37 288
7 0


In [26]:
#Player strategies
#Hand probabilities
#Bluffing chance
#Chance hand will be higher than others
#Creedence of other players - probability others are bluffing